<a href="https://colab.research.google.com/github/davidlealo/aprendiendolangchain/blob/main/libros_extensos_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalación de bibliotecas necesarias
!pip install langchain openai PyPDF2 faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 16.5 MB/s eta 0:00:00


In [ ]:
# Librerías necesarias
from google.colab import files
import getpass
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Subir la API Key de OpenAI
api_key = getpass.getpass("Por favor, introduce tu API key de OpenAI: ")
import os
os.environ["OPENAI_API_KEY"] = api_key

# Subir archivo PDF
print("Sube el libro en formato PDF:")
uploaded = files.upload()

# Verificar archivo subido
file_name = list(uploaded.keys())[0]
print(f"Archivo cargado: {file_name}")

# Cargar el libro
loader = PyPDFLoader(file_name)
pages = loader.load()

# Dividir el libro en fragmentos
print("Dividiendo el libro en fragmentos manejables...")
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = splitter.split_documents(pages)
print(f"Se dividió el libro en {len(documents)} fragmentos.")

# Generar embeddings para los fragmentos
print("Generando embeddings...")
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts([doc.page_content for doc in documents], embeddings)

# Guardar la base de datos vectorial en la memoria
vectorstore.save_local("vectorstore_faiss")
print("Base de datos vectorial creada y almacenada.")

# Crear la cadena de preguntas y respuestas
print("Configurando el sistema de consultas...")
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(model="gpt-4"),
    retriever=vectorstore.as_retriever(),
)

# Preguntar al usuario
print("\nAhora puedes hacer preguntas sobre el libro.")
while True:
    pregunta = input("\nEscribe tu pregunta (o 'salir' para terminar): ")
    if pregunta.lower() == "salir":
        print("¡Gracias por usar el analizador de libros!")
        break
    respuesta = qa_chain.run(pregunta)
    print(f"\nRespuesta:\n{respuesta}")

# Análisis avanzado (resúmenes automáticos)
def generar_resumen():
    print("\nGenerando un resumen del libro...")
    prompt = PromptTemplate(
        input_variables=["texto"],
        template="Por favor, resume el siguiente texto:\n\n{texto}",
    )
    summary_chain = LLMChain(llm=OpenAI(model="gpt-4"), prompt=prompt)
    resumen = summary_chain.run(documents[0].page_content)  # Resumir el primer fragmento
    print(f"\nResumen del primer fragmento:\n{resumen}")

# Opcional: Resúmenes
generar_resumen()


## Nuevo enfoque

In [3]:
# Instalación de bibliotecas necesarias
!pip install langchain openai PyPDF2 faiss-cpu ebooklib beautifulsoup4 lxml



In [5]:
!pip install --upgrade langchain langchain-openai langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchai

In [ ]:
# Librerías necesarias
from google.colab import files
import getpass
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from PyPDF2 import PdfReader
from ebooklib import epub
from bs4 import BeautifulSoup

# Subir la API Key de OpenAI
api_key = getpass.getpass("Por favor, introduce tu API key de OpenAI: ")
os.environ["OPENAI_API_KEY"] = api_key

# Subir archivo (PDF o EPUB)
print("Sube el libro en formato PDF o EPUB:")
uploaded = files.upload()

# Obtener el nombre del archivo
file_name = list(uploaded.keys())[0]
print(f"Archivo cargado: {file_name}")

# Función para leer PDF
def leer_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Función para leer EPUB
def leer_epub(file_path):
    book = epub.read_epub(file_path)
    text = ""
    for item in book.get_items():
        if item.get_type() == epub.ITEM_DOCUMENT:
            soup = BeautifulSoup(item.get_content(), "html.parser")
            text += soup.get_text()
    return text

# Determinar el tipo de archivo y cargar el texto
if file_name.lower().endswith(".pdf"):
    print("Procesando archivo PDF...")
    contenido = leer_pdf(file_name)
elif file_name.lower().endswith(".epub"):
    print("Procesando archivo EPUB...")
    contenido = leer_epub(file_name)
else:
    raise ValueError("Formato no soportado. Por favor, sube un archivo PDF o EPUB.")

# Dividir el contenido en fragmentos
print("Dividiendo el libro en fragmentos manejables...")
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = splitter.split_text(contenido)
print(f"Se dividió el libro en {len(documents)} fragmentos.")

# Generar embeddings para los fragmentos
print("Generando embeddings...")
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(documents, embeddings)

# Guardar la base de datos vectorial en la memoria
vectorstore.save_local("vectorstore_faiss")
print("Base de datos vectorial creada y almacenada.")

# Crear la cadena de preguntas y respuestas
print("Configurando el sistema de consultas...")
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(model="gpt-4"),
    retriever=vectorstore.as_retriever(),
)

# Preguntar al usuario
print("\nAhora puedes hacer preguntas sobre el libro.")
while True:
    pregunta = input("\nEscribe tu pregunta (o 'salir' para terminar): ")
    if pregunta.lower() == "salir":
        print("¡Gracias por usar el analizador de libros!")
        break
    respuesta = qa_chain.run(pregunta)
    print(f"\nRespuesta:\n{respuesta}")

# Análisis avanzado (resúmenes automáticos)
def generar_resumen():
    print("\nGenerando un resumen del libro...")
    prompt = PromptTemplate(
        input_variables=["texto"],
        template="Por favor, resume el siguiente texto:\n\n{texto}",
    )
    summary_chain = LLMChain(llm=OpenAI(model="gpt-4"), prompt=prompt)
    resumen = summary_chain.run(documents[0])  # Resumir el primer fragmento
    print(f"\nResumen del primer fragmento:\n{resumen}")

# Opcional: Generar resúmenes
generar_resumen()


In [8]:
# Instalación de bibliotecas necesarias
!pip install langchain langchain-openai openai PyPDF2 faiss-cpu ebooklib beautifulsoup4 lxml

# Librerías necesarias
from google.colab import files
import getpass
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from PyPDF2 import PdfReader
from ebooklib import epub
from bs4 import BeautifulSoup

# Subir la API Key de OpenAI
api_key = getpass.getpass("Por favor, introduce tu API key de OpenAI: ")
os.environ["OPENAI_API_KEY"] = api_key

# Subir archivo (PDF o EPUB)
print("Sube el libro en formato PDF o EPUB:")
uploaded = files.upload()

# Obtener el nombre del archivo
file_name = list(uploaded.keys())[0]
print(f"Archivo cargado: {file_name}")

# Función para leer PDF
def leer_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Función para leer EPUB con corrección
def leer_epub(file_path):
    book = epub.read_epub(file_path)
    text = ""

    # Procesar los ítems del libro
    for item in book.get_items():
        # Verificar si el contenido es de tipo XHTML
        if item.media_type == "application/xhtml+xml":
            soup = BeautifulSoup(item.get_content(), "html.parser")
            text += soup.get_text()

    return text

# Determinar el tipo de archivo y cargar el texto
if file_name.lower().endswith(".pdf"):
    print("Procesando archivo PDF...")
    contenido = leer_pdf(file_name)
elif file_name.lower().endswith(".epub"):
    print("Procesando archivo EPUB...")
    contenido = leer_epub(file_name)
else:
    raise ValueError("Formato no soportado. Por favor, sube un archivo PDF o EPUB.")

# Dividir el contenido en fragmentos
print("Dividiendo el libro en fragmentos manejables...")
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = splitter.split_text(contenido)
print(f"Se dividió el libro en {len(documents)} fragmentos.")

# Generar embeddings para los fragmentos
print("Generando embeddings...")
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(documents, embeddings)

# Guardar la base de datos vectorial en la memoria
vectorstore.save_local("vectorstore_faiss")
print("Base de datos vectorial creada y almacenada.")

# Crear la cadena de preguntas y respuestas
print("Configurando el sistema de consultas...")
retriever = vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-4", temperature=0.7),  # Uso explícito de ChatOpenAI
    retriever=retriever,
)

# Preguntar al usuario
print("\nAhora puedes hacer preguntas sobre el libro.")
while True:
    pregunta = input("\nEscribe tu pregunta (o 'salir' para terminar): ")
    if pregunta.lower() == "salir":
        print("¡Gracias por usar el analizador de libros!")
        break
    respuesta = qa_chain.run({"query": pregunta})  # Ajuste para pasar la consulta como diccionario
    print(f"\nRespuesta:\n{respuesta}")

# Análisis avanzado (resúmenes automáticos)
def generar_resumen():
    print("\nGenerando un resumen del libro...")
    prompt = PromptTemplate(
        input_variables=["texto"],
        template="Por favor, resume el siguiente texto:\n\n{texto}",
    )
    summary_chain = LLMChain(llm=ChatOpenAI(model="gpt-4"), prompt=prompt)
    resumen = summary_chain.run({"texto": documents[0]})  # Resumir el primer fragmento
    print(f"\nResumen del primer fragmento:\n{resumen}")

# Opcional: Generar resúmenes
generar_resumen()


Por favor, introduce tu API key de OpenAI: ··········
Sube el libro en formato PDF o EPUB:


Saving Obras Completas de Aristóteles (Aristóteles) (Z-Library).epub to Obras Completas de Aristóteles (Aristóteles) (Z-Library) (2).epub
Archivo cargado: Obras Completas de Aristóteles (Aristóteles) (Z-Library) (2).epub
Procesando archivo EPUB...


/usr/local/lib/python3.10/dist-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/usr/local/lib/python3.10/dist-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


Dividiendo el libro en fragmentos manejables...
Se dividió el libro en 8307 fragmentos.
Generando embeddings...
Base de datos vectorial creada y almacenada.
Configurando el sistema de consultas...

Ahora puedes hacer preguntas sobre el libro.

Escribe tu pregunta (o 'salir' para terminar): cuantos textos distints tiene el libro?

Respuesta:
La información proporcionada no especifica el número total de textos o capítulos en el libro.

Escribe tu pregunta (o 'salir' para terminar): resumen la etica a nicomaco

Respuesta:
Lo siento, pero no se proporciona suficiente información en el contexto proporcionado para proporcionar un resumen completo de "Ética a Nicómaco".

Escribe tu pregunta (o 'salir' para terminar): salir
¡Gracias por usar el analizador de libros!

Generando un resumen del libro...


<ipython-input-8-09ea0d2cb3b6>:104: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  summary_chain = LLMChain(llm=ChatOpenAI(model="gpt-4"), prompt=prompt)



Resumen del primer fragmento:
Este texto parece ser el índice de una edición de las Obras Completas de Aristóteles, traducida por Patricio de Azcárate. Se mencionan varias obras importantes de Aristóteles, incluyendo "Moral a Nicómaco", "Política", "Tratado del alma", "Metafísica" y otros. La edición fue publicada por Ebooklasicos, con la primera edición en diciembre de 2015 y la última en diciembre de 2020. También se hace referencia a una biografía de Aristóteles y del traductor, Patricio de Azcárate.
